<a href="https://colab.research.google.com/github/colinjhorgan/MLTSAFinalProject/blob/main/Preprocessing/CNN_Preprocessing_Cart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as plt
%pylab inline

from scipy.signal import resample_poly

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import cross_val_score

from google.colab import drive
drive.mount('/content/gdrive')

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Mounted at /content/gdrive


In [ ]:
#Read in data
path = '/content/gdrive/MyDrive/3. MSDS Courses/Machine Learning Time Series/MLTSA22 Project/Data/Phones_accelerometer.csv'
paccel = pd.read_csv(path, index_col=0)

paccel.tail(8)

FileNotFoundError: ignored

In [ ]:
#gt contains nan values so we impute them with 'no_task'
paccel['gt'] = paccel['gt'].fillna('no_task')

In [ ]:
#Resample all trials to 100Hz
x_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['x'].apply(lambda x : resample_poly(x.values, 15050, x.values.shape[0]))
y_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['y'].apply(lambda y : resample_poly(y.values, 15050, y.values.shape[0]))
z_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['z'].apply(lambda z : resample_poly(z.values, 15050, z.values.shape[0]))

#Re-construct
new_df = x_fixed.to_frame()
new_df = new_df.merge(y_fixed.to_frame(), left_index=True, right_index=True)
new_df = new_df.merge(z_fixed.to_frame(), left_index=True, right_index=True)

In [ ]:
x_clip, y_clip, z_clip = [], [], []

for i in range(0, new_df.shape[0]):
    
    sub = new_df[['x','y','z']].iloc[i,:]
    
    x_clip.append(sub['x'].clip(-20,20))
    y_clip.append(sub['y'].clip(-20,20))
    z_clip.append(sub['z'].clip(-20,20))

In [ ]:
#Add to new_df
new_df['x_clip'] = x_clip
new_df['y_clip'] = y_clip
new_df['z_clip'] = z_clip

In [ ]:
#Standardize each series, and apply sliding window with length 100 and 50% overlap

#sig = lambda x : 1/(1 + np.exp(-x)) # sigmoid function

images = []
for i in range(new_df.shape[0]):
    x = new_df.iloc[[i]].x.values[0]
    y = new_df.iloc[[i]].y.values[0]
    z = new_df.iloc[[i]].z.values[0]

    x = ((x - x.min() ) / (x.max() - x.min()))
    y = ((y - y.min() ) / (y.max() - y.min()))
    z = ((z - z.min() ) / (z.max() - z.min()))
    
    r = np.stack([x[i:100+i] for i in range(0, 15000, 50)]) # stack elements using 100-sample window with 50% overlap
    g = np.stack([y[i:100+i] for i in range(0, 15000, 50)])
    b = np.stack([z[i:100+i] for i in range(0, 15000, 50)])
    
    vals = np.dstack([r,g,b])
    images.append(vals)
    
im_arr = np.array(images)

In [ ]:
#Standardize each series, and apply sliding window with length 100 and 50% overlap

#sig = lambda x : 1/(1 + np.exp(-x)) # sigmoid function

images = []
for i in range(new_df.shape[0]):
    x = new_df.iloc[[i]].x_clip.values[0]
    y = new_df.iloc[[i]].y_clip.values[0]
    z = new_df.iloc[[i]].z_clip.values[0]

    x = ((x + 20 ) / 40)
    y = ((y + 20 ) / 40)
    z = ((z + 20 ) / 40)
    
    r = np.stack([x[i:100+i] for i in range(0, 15000, 50)]) # stack elements using 100-sample window with 50% overlap
    g = np.stack([y[i:100+i] for i in range(0, 15000, 50)])
    b = np.stack([z[i:100+i] for i in range(0, 15000, 50)])
    
    vals = np.dstack([r,g,b])
    images.append(vals)
    
im_arr = np.array(images)

In [ ]:
im_arr.shape #(num_samples, height, length, num_layers)

(485, 300, 100, 3)

In [ ]:
np.save('/content/gdrive/MyDrive/3. MSDS Courses/Machine Learning Time Series/MLTSA22 Project/Data/image_arrayminmax.npy', im_arr)